<div style='background-image:url("https://tinyurl.com/587zdbkm/title01.png") ; padding: 0px ; background-size: cover ; border-radius: 5px ; height: 200px'>
<div style="float: right ; margin: 50px ; padding: 20px ; background: rgba(255 , 255 , 255 , 0.7) ; width: 50% ; height: 150px">
<div style="position: relative ; top: 50% ; transform: translatey(-50%)">
            <div style="font-size: xx-large ; font-weight: 900 ; color: rgba(0 , 0 , 0 , 0.8) ; line-height: 100%">Computers, Waves, Simulations</div>
            <div style="font-size: large ; padding-top: 20px ; color: rgba(0 , 0 , 0 , 0.5)">Finite-Difference Method - Acoustic Waves 1D</div>
        </div>
    </div>
</div>

#### This notebook covers the following aspects:

* Implementation of the 1D acoustic wave equation 
* Understanding the input parameters for the simulation and the plots that are generated
* Understanding the concepts of stability (Courant criterion)
* Modifying source and receiver locations and observing the effects on the seismograms

**Note:** Corrections implemented May 14, 2020
* Error in CFL criterion fixed
* Error in source time function fixed


#### Numerical Solution (Finite Differences Method)

The acoustic wave equation in 1D with constant density 

$$
\partial^2_t p(x,t) \ = \ c(x)^2 \partial_x^2 p(x,t) + s(x,t)
$$

with pressure $p$, acoustic velocity $c$, and source term $s$ contains two second derivatives that can be approximated with a difference formula such as

$$
\partial^2_t p(x,t) \ \approx \ \frac{p(x,t+dt) - 2 p(x,t) + p(x,t-dt)}{dt^2} 
$$

and equivalently for the space derivative. Injecting these approximations into the wave equation allows us to formulate the pressure p(x) for the time step $t+dt$ (the future) as a function of the pressure at time $t$ (now) and $t-dt$ (the past). This is called an explicit scheme allowing the $extrapolation$ of the space-dependent field into the future only looking at the nearest neighbourhood.

We replace the time-dependent (upper index time, lower indices space) part by

$$
 \frac{p_{i}^{n+1} - 2 p_{i}^n + p_{i}^{n-1}}{\mathrm{d}t^2} \ = \ c^2 ( \partial_x^2 p) \ + s_{i}^n
$$

solving for $p_{i}^{n+1}$.

The extrapolation scheme is

$$
p_{i}^{n+1} \ = \ c_i^2 \mathrm{d}t^2 \left[ \partial_x^2 p \right]
+ 2p_{i}^n - p_{i}^{n-1} + \mathrm{d}t^2 s_{i}^n
$$

The  space derivatives are determined by 

$$
\partial_x^2 p \ = \ \frac{p_{i+1}^{n} - 2 p_{i}^n + p_{i-1}^{n}}{\mathrm{d}x^2}
$$

#### Exercises: 

* Increase the dominant frequency f0 and observe how the fit with the analytical solution gets worse. Fix the frequency and change nop to 5 (longer and more accurate space derivative). Observe how the solution improves comapared to nop = 3!

* Increase dt by a factor of 2 (keeping everything else) and observe how the solution becomes unstable. 

In [ ]:
# Import Libraries
# ----------------------------------------------
import numpy as np
import plotly.graph_objects as go

from plotly.subplots import make_subplots

# Ignore Warning Messages
# -----------------------
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Parameter Configuration 
# -----------------------

nx   = 10000        # number of grid points in x-direction
xmax = 10000        # physical domain (m)
dx   = xmax/(nx-1)  # grid point distance in x-direction
c0   = 334.         # wave speed in medium (m/s)
isrc = int(nx/2)    # source location in grid in x-direction
ir   = isrc + 100          # receiver location in grid in x-direction
nt   = 600          # maximum number of time steps
dt   = 0.002        # time step
op   = 3            # Length of FD operator for 2nd space derivative (3 or 5) 
print(op, '- point operator')

# Source time function parameters
f0   = 15. # dominant frequency of the source (Hz)
t0   = 4. / f0 # source time shift

# CPL Stability Criterion
# --------------------------
eps = c0 * dt / dx #epsilon value (corrected May 3, 2020)

print('Stability criterion =', eps)

In [ ]:
# Source time function (Gaussian)
# -------------------------------
src  = np.zeros(nt + 1)
time = np.linspace(0 * dt, nt * dt, nt)
# 1st derivative of a Gaussian (corrected May 3, 2020)
src  = -8. * (time - t0) * f0 * (np.exp(-1.0 * (4*f0) ** 2 * (time - t0) ** 2))



In [ ]:

# Create subplot with 1 row and 2 columns
fig = make_subplots(rows=1, cols=2)

# Add source time function to subplot
fig.add_trace(
    go.Scatter(x=time, y=src, mode='lines', name='Source Time Function'),
    row=1, col=1
)

# Add source spectrum to subplot
spec = np.fft.fft(src) # source time function in frequency domain
freq = np.fft.fftfreq(spec.size, d = dt ) # time domain to frequency domain
fig.add_trace(
    go.Scatter(x=np.abs(freq), y=np.abs(spec), mode='lines', name='Source Spectrum'),
    row=1, col=2
)

# Update xaxis properties
fig.update_xaxes(title_text="Time (s)", row=1, col=1)
fig.update_xaxes(title_text="Frequency (Hz)", range=[0, 250], row=1, col=2)

# Update yaxis properties
fig.update_yaxes(title_text="Amplitude", row=1, col=1)
fig.update_yaxes(title_text="Amplitude", row=1, col=2)

# Update layout and title
fig.update_layout(height=600, width=1000, title_text="Source Time Function and Source Spectrum")

fig.show()

In [ ]:
# Plot Snapshot & Seismogram - Run this code after simulation 
# ---------------------------------------------------------------------------

# Initialize empty pressure
# -------------------------
p    = np.zeros(nx) # p at time n (now)
pold = np.zeros(nx) # p at time n-1 (past)
pnew = np.zeros(nx) # p at time n+1 (present)
d2px = np.zeros(nx) # 2nd space derivative of p

# Initialize model (assume homogeneous model)
# -------------------------------------------
c    = np.zeros(nx)
c    = c + c0       # initialize wave velocity in model

# Initialize coordinate
# ---------------------
x    = np.arange(nx)
x    = x * dx       # coordinate in x-direction

# Initialize empty seismogram
# ---------------------------
seis = np.zeros(nt) 

# Analytical solution
# -------------------
G    = time * 0.
for it in range(nt): # Calculate Green's function (Heaviside function)
    if (time[it] - np.abs(x[ir] - x[isrc]) / c0) >= 0:
        G[it] = 1. / (2 * c0)
Gc   = np.convolve(G, src * dt)
Gc   = Gc[0:nt]
lim  = Gc.max() # get limit value from the maximum amplitude


# Create subplot with 1 row and 2 columns
fig = make_subplots(rows=1, cols=2)

# Add 1D wave propagation to subplot
fig.add_trace(
    go.Scatter(x=[isrc], y=[0], mode='markers', marker=dict(color='red', size=11, symbol='star'), name='Source'),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=[ir], y=[0], mode='markers', marker=dict(color='black', size=8, symbol='triangle-up'), name='Receiver'),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=list(range(len(p))), y=p, mode='lines', name='Pressure Amplitude'),
    row=1, col=1
)

# Add seismogram to subplot
fig.add_trace(
    go.Scatter(x=[0], y=[0], mode='lines', line=dict(color='red', dash='dash'), name='Analytical'),
    row=1, col=2
)
fig.add_trace(
    go.Scatter(x=[0], y=[0], mode='lines', line=dict(color='blue'), name='FD'),
    row=1, col=2
)
fig.add_trace(
    go.Scatter(x=time, y=seis, mode='lines', name='Amplitude'),
    row=1, col=2
)
fig.add_trace(
    go.Scatter(x=[0], y=[0], mode='markers', marker=dict(color='red', size=15, symbol='line-ns'), name='Time Step Position'),
    row=1, col=2
)
fig.add_trace(
    go.Scatter(x=time, y=Gc, mode='lines', line=dict(color='red', dash='dash'), name='Analytical Solution'),
    row=1, col=2
)

# Update xaxis properties
fig.update_xaxes(title_text="x (m)", range=[0, xmax], row=1, col=1)
fig.update_xaxes(title_text="Time (s)", range=[time[0], time[-1]], row=1, col=2)

# Update yaxis properties
fig.update_yaxes(title_text="Pressure Amplitude", range=[-np.max(p), np.max(p)], row=1, col=1)
fig.update_yaxes(title_text="Amplitude", row=1, col=2)

# Update layout and title
fig.update_layout(height=600, width=1000, title_text="1D Wave Propagation and Seismogram")

fig.show()

In [ ]:
# Create subplot with 1 row and 2 columns
fig = make_subplots(rows=1, cols=2)

# Add animation trace
fig.add_trace(
    go.Scatter(x=list(range(len(p))), y=p,
               mode='lines', name='Pressure Amplitude'),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=time, y=seis, mode='lines', line=dict(color='cyan'), name='Amplitude'),
    row=1, col=2
)

# Add 1D wave propagation to subplot
fig.add_trace(
    go.Scatter(x=[isrc], y=[0], mode='markers', marker=dict(
        color='red', size=11, symbol='star'), name='Source'),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=[ir], y=[0], mode='markers', marker=dict(
        color='black', size=8, symbol='triangle-up'), name='Receiver'),
    row=1, col=1
)

# Add seismogram to subplot
fig.add_trace(
    go.Scatter(x=[0], y=[0], mode='lines', line=dict(
        color='red', dash='dash'), name='Analytical'),
    row=1, col=2
)
fig.add_trace(
    go.Scatter(x=[0], y=[0], mode='lines', line=dict(color='blue'), name='FD'),
    row=1, col=2
)
fig.add_trace(
    go.Scatter(x=[0], y=[0], mode='markers', marker=dict(
        color='red', size=15, symbol='line-ns'), name='Time Step Position'),
    row=1, col=2
)
fig.add_trace(
    go.Scatter(x=time, y=Gc, mode='lines', line=dict(
        color='red', dash='dash'), name='Analytical Solution'),
    row=1, col=2
)


# Calculate Partial Derivatives
max_p = -np.inf
min_p = np.inf
frames = []
x_t = []
for it in range(nt):
    if op == 3:  # use 3 point operator FD scheme
        for i in range(1, nx - 1):
            d2px[i] = (p[i + 1] - 2 * p[i] + p[i - 1]) / dx ** 2

    if op == 5:  # use 5 point operator FD scheme
        for i in range(2, nx - 2):
            d2px[i] = (-1./12 * p[i + 2] + 4./3 * p[i + 1] - 5./2 * p[i]
                       + 4./3 * p[i - 1] - 1./12 * p[i - 2]) / dx ** 2

    # Time Extrapolation
    # ------------------
    pnew = 2 * p - pold + c ** 2 * dt ** 2 * d2px

    # Solution in space-time
    x_t.append(pnew)

    # Add Source Term at isrc
    # -----------------------
    # Absolute pressure w.r.t analytical solution
    pnew[isrc] = pnew[isrc] + src[it] / (dx) * dt ** 2

    # Remap Time Levels
    # -----------------
    pold, p = p, pnew
    if p.max() > max_p: max_p = p.max()
    if p.min() < min_p: min_p = p.min()

    # Output Seismogram
    # -----------------
    seis[it] = p[ir]

    # Plot pressure field
    # -------------------------------------
    # Snapshot
    idisp = 5  # display frequency
    if (it % idisp) == 0:
        frames.append(go.Frame(data=[go.Scatter(x=list(range(len(p))), y=p,
                                                mode='lines', name='Pressure Amplitude'),
                                     go.Scatter(x=time[:it], y=seis[:it], mode='lines', name='Amplitude')]))


# Update xaxis properties
fig.update_xaxes(title_text="x (m)", range=[0, xmax], row=1, col=1)
fig.update_xaxes(title_text="Time (s)", range=[
                 time[0], time[-1]], row=1, col=2)

# Update yaxis properties
fig.update_yaxes(title_text="Pressure Amplitude", range=[min_p, max_p], row=1, col=1)
fig.update_yaxes(title_text="Amplitude", row=1, col=2)


# Update the original figure with frames
fig.frames = frames


# Update layout with the slider and the button
fig.update_layout(
    height=600, width=1000,
    title_text="1D Wave Propagation and Seismogram",
    updatemenus=[dict(
        type="buttons",
        showactive=False,
        buttons=[dict(label="Play",
                      method="animate",
                      args=[None, {"frame": {"duration": 0, "redraw": True},
                                   "fromcurrent": True, "transition": {"duration": 0}}],
                      ),
                  dict(label="Pause",
                       method="animate",
                       args=[[None], {"frame": {"duration": 0, "redraw": False},
                                      "mode": "immediate",
                                      "transition": {"duration": 0}}],
                       )
                  ]
    )]
)

fig.show()

In [ ]:
# Solution in space-time
x_t = np.asanyarray(x_t)

# Initialize plot
fig = go.Figure(data=go.Heatmap(
    z=x_t,
    x=np.linspace(0, xmax, x_t.shape[1]),
    y=np.linspace(0, nt*dt, x_t.shape[0]),
    colorscale='Greys',
    showscale=False))

fig.update_layout(title='u(x,t) Wavefield', 
                  xaxis_title='Space [m]', 
                  yaxis_title='Time [s]',
                  yaxis_autorange='reversed')

fig.show()